<a href="https://colab.research.google.com/github/hsuecu/data_set/blob/main/ADULT_CENSUS_PSO_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<span style="display:inline-block;position:relative;font-size:64px;background-color: #d9edab;padding:200px 10px;width:100%;text-align:center">ADULT CENSUS</span><br><strong style="display:block;font-size:30px;padding-top:20px">GA</strong>

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time as time

!pip3 install scikit-learn-intelex

from sklearnex import patch_sklearn
patch_sklearn()

import warnings
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [28]:
data = pd.read_csv("https://raw.githubusercontent.com/hsuecu/python_ml/main/adult.csv")
data.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [30]:
if __name__=="__main__":
    for i in data.columns:
        data[i].replace('?', np.NAN, inplace=True)

In [31]:
if __name__=="__main__":
    count = 0
    for i in range(data.shape[0]):
        for j in data.iloc[i].isna():
            if j:
                count += 1
                break
    print(f'Useless row entries are:{round(count/data.shape[0] * 100, 3)}%')

Useless row entries are:7.368%


In [32]:
data.dropna(axis=0, inplace=True)

In [33]:
data.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30162 entries, 1 to 32560
Columns: 15 entries, age to income
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


# Data Processing

In [34]:
to_oneHotEncode = ['race', 'sex']
to_lableEncode = ['workclass', 'education',
                 'marital.status', 'occupation',
                 'relationship', 'native.country']

data_intermediate=pd.get_dummies(data=data[to_oneHotEncode],
                                drop_first=False)
data_intermediate.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30162 entries, 1 to 32560
Columns: 7 entries, race_Amer-Indian-Eskimo to sex_Male
dtypes: uint8(7)
memory usage: 441.8 KB


In [35]:
data = pd.concat([data, data_intermediate], axis = 1)
data.drop(to_oneHotEncode, axis=1, inplace=True)

In [36]:
from sklearn.preprocessing import LabelEncoder
le_obj = LabelEncoder()

if __name__=="__main__":
    for i in to_lableEncode:
        data[i] = le_obj.fit_transform(data[i])

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30162 entries, 1 to 32560
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   age                      30162 non-null  int64 
 1   workclass                30162 non-null  int64 
 2   fnlwgt                   30162 non-null  int64 
 3   education                30162 non-null  int64 
 4   education.num            30162 non-null  int64 
 5   marital.status           30162 non-null  int64 
 6   occupation               30162 non-null  int64 
 7   relationship             30162 non-null  int64 
 8   capital.gain             30162 non-null  int64 
 9   capital.loss             30162 non-null  int64 
 10  hours.per.week           30162 non-null  int64 
 11  native.country           30162 non-null  int64 
 12  income                   30162 non-null  object
 13  race_Amer-Indian-Eskimo  30162 non-null  uint8 
 14  race_Asian-Pac-Islander  30162 non-nul

In [38]:
data['income'] = data['income'].apply(lambda x:1 if x=='>50K' else 0)

In [39]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
col = [i for i in data
       if i not in ['race_Amer-Indian-Eskimo',
        'race_Asian-Pac-Islander','race_Black','race_Other',
        'race_White','sex_Female','sex_Male', 'income']]

In [40]:
col

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education.num',
 'marital.status',
 'occupation',
 'relationship',
 'capital.gain',
 'capital.loss',
 'hours.per.week',
 'native.country']

In [41]:
X = data.drop(['income'], axis=1)
y = data['income']

print(f'X is {X.shape}\nY is {y.shape}')

X[col] = scaler.fit_transform(X[col])

X is (30162, 19)
Y is (30162,)


In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, train_size=0.7)

In [43]:
print(f'X_train:{X_train.shape}\n\
X_test:{X_test.shape}\n\
y_train:{y_train.shape}\n\
y_test:{y_test.shape}')

X_train:(21113, 19)
X_test:(9049, 19)
y_train:(21113,)
y_test:(9049,)


In [44]:
# function to evaluate
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score, average_precision_score
from sklearn.model_selection import StratifiedKFold
def analyse(model, xtrain, ytrain, xtest, ytest):
    ytrain_pred = model.predict(xtrain)
    ytest_pred = model.predict(xtest)
    try:
        y_score_test = model.decision_function(xtest)
        y_score_train = model.decision_function(xtrain)
    except:
        y_score_test = model.predict_proba(xtest)[:,1]
        y_score_train = model.predict_proba(xtrain)[:,1]

    ret = {'train':{'pr auc': average_precision_score(ytrain, y_score_train),
                'accuracy': accuracy_score(ytrain, ytrain_pred),
                'recall':recall_score(ytrain, ytrain_pred),
                'precision':precision_score(ytrain, ytrain_pred),
                'roc auc':roc_auc_score(ytrain, y_score_train),
                'f1 score': f1_score(ytrain, ytrain_pred)},
           'test':{'pr auc': average_precision_score(ytest, y_score_test),
                'accuracy':accuracy_score(ytest, ytest_pred),
                'recall':recall_score(ytest, ytest_pred),
                'precision':precision_score(ytest, ytest_pred),
                'roc auc':roc_auc_score(ytest, y_score_test),
                'f1 score': f1_score(ytest, ytest_pred)}}
    return ret

In [45]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score, auc, precision_recall_curve

def analyseANN(model, xtrain, ytrain, xtest, ytest):
    ypred =  model.predict(xtrain)
    ypred_test = model.predict(xtest)
    precision_train, recall_train, thresholds = precision_recall_curve(ytrain, ypred)
    precision_test, recall_test, thresholds = precision_recall_curve(y_test, ypred_test)

    for i in range(len(ypred)):
        if ypred[i] <=0.5:
            ypred[i] =0
        else: 
            ypred[i] =1
    for i in range(len(ypred_test)):
        if ypred_test[i] <=0.5:
            ypred_test[i] =0
        else: 
            ypred_test[i] =1
    accuracy = accuracy_score(ytrain, ypred)
    recall = recall_score(ytrain, ypred)
    precision = precision_score(ytrain, ypred)
    roc_auc = roc_auc_score(ytrain, ypred)
    f1score = f1_score(ytrain, ypred)

    ret = {'train':{'pr auc': auc(recall_train, precision_train),
                    'accuracy': accuracy,
                    'recall':recall,
                    'precision':precision,
                    'roc auc':roc_auc,
                    'f1 score': f1score},
               'test':{'pr auc': auc(recall_test, precision_test),
                    'accuracy':accuracy_score(ytest, ypred_test),
                    'recall':recall_score(ytest, ypred_test),
                    'precision':precision_score(ytest, ypred_test),
                    'roc auc':roc_auc_score(ytest, ypred_test),
                    'f1 score': f1_score(ytest, ypred_test)}}
    return ret

# BASE CASE

In [46]:
##                     ##
## LOGISTIC REGRESSION ##
##                     ##

from sklearn.linear_model import LogisticRegression
import json
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)
print(json.dumps(analyse(lr, X_train, y_train, X_test, y_test), indent=2))

{
  "train": {
    "pr auc": 0.6916993805219034,
    "accuracy": 0.818974091791787,
    "recall": 0.4604261796042618,
    "precision": 0.7105108631826189,
    "roc auc": 0.8476919898821746,
    "f1 score": 0.5587624105287463
  },
  "test": {
    "pr auc": 0.7019601996037149,
    "accuracy": 0.8261686374184993,
    "recall": 0.4911190053285968,
    "precision": 0.7214611872146118,
    "roc auc": 0.8540344436776125,
    "f1 score": 0.5844121532364597
  }
}


In [47]:
##                     ##
## XG BOOST CLASSIFIER ##
##                     ##

from xgboost.sklearn import XGBClassifier
params = {
            'booster':'gbtree',
            'objective':'binary:logistic',
            'max_depth': 3,
            'learning_rate': 1,
            'n_estimators':100,
            "eval_metric": ['logloss'],
            'random_state': 42
        }         
           
          
# instantiate the classifier 
xgb_clf = XGBClassifier(**params)
xgb_clf.fit(X_train, y_train)
print(json.dumps(analyse(xgb_clf, X_train, y_train, X_test, y_test), indent=2))

{
  "train": {
    "pr auc": 0.8826203421971026,
    "accuracy": 0.8937147728887416,
    "recall": 0.730593607305936,
    "precision": 0.8226221079691517,
    "roc auc": 0.9512246726810365,
    "f1 score": 0.773881499395405
  },
  "test": {
    "pr auc": 0.8103428359030254,
    "accuracy": 0.8593214719858548,
    "recall": 0.6642984014209592,
    "precision": 0.7431693989071039,
    "roc auc": 0.9153822956580729,
    "f1 score": 0.7015240328253225
  }
}


In [48]:
##                          ##
## RANDOM FOREST CLASSIFIER ##
##                          ##


from sklearn.ensemble import RandomForestClassifier
rforest = RandomForestClassifier(max_depth = 3)
rforest.fit(X_train, y_train)
print(json.dumps(analyse(rforest, X_train, y_train, X_test, y_test), indent=2))

{
  "train": {
    "pr auc": 0.7615936139053209,
    "accuracy": 0.8237578742954578,
    "recall": 0.3386605783866058,
    "precision": 0.8790123456790123,
    "roc auc": 0.8924816501151032,
    "f1 score": 0.4889438263974729
  },
  "test": {
    "pr auc": 0.764800187712424,
    "accuracy": 0.8266106752127307,
    "recall": 0.3552397868561279,
    "precision": 0.8724100327153762,
    "roc auc": 0.8928284628758221,
    "f1 score": 0.5048911328494794
  }
}


In [49]:
##                        ##
## NAIVE BAYES CLASSIFIER ##
##                        ##


from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
print(json.dumps(analyse(gnb, X_train, y_train, X_test, y_test), indent=2))

{
  "train": {
    "pr auc": 0.6757307953318539,
    "accuracy": 0.7988443139298063,
    "recall": 0.3611111111111111,
    "precision": 0.6810190168640115,
    "roc auc": 0.8435924879024854,
    "f1 score": 0.4719631978117618
  },
  "test": {
    "pr auc": 0.6850802147991403,
    "accuracy": 0.8028511437727925,
    "recall": 0.37211367673179396,
    "precision": 0.6937086092715232,
    "roc auc": 0.8477097564984657,
    "f1 score": 0.48439306358381495
  }
}


In [50]:
##                ##
## SVC CLASSIFIER ##
##                ##


from sklearn.svm import SVC
svc = SVC(C=1, kernel='rbf', gamma='scale')
svc.fit(X_train, y_train)
print(json.dumps(analyse(svc, X_train, y_train, X_test, y_test), indent=2))

{
  "train": {
    "pr auc": 0.6566129341806218,
    "accuracy": 0.7954814569222753,
    "recall": 0.2720700152207002,
    "precision": 0.7440166493236212,
    "roc auc": 0.8292357031052551,
    "f1 score": 0.3984396767901923
  },
  "test": {
    "pr auc": 0.6620484878410027,
    "accuracy": 0.7972151618963421,
    "recall": 0.2793072824156306,
    "precision": 0.7479191438763377,
    "roc auc": 0.8302868311717294,
    "f1 score": 0.4067248625929518
  }
}


In [51]:
##                ##
## ANN CLASSIFIER ##
##                ##


import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

def Model():
    ann = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=10, activation='relu',solver='adam')
    return ann

In [52]:
_start = time.time()

model= Model()

model.fit(X_train, y_train)

ret = analyseANN(model, X_train, y_train, X_test, y_test)

_end = time.time()

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')
print(json.dumps(ret, indent=2))

running for: 0m, 4s
{
  "train": {
    "pr auc": 0.6459090175040106,
    "accuracy": 0.8096907118836736,
    "recall": 0.5348173515981736,
    "precision": 0.6411952554744526,
    "roc auc": 0.7178091298572314,
    "f1 score": 0.583195020746888
  },
  "test": {
    "pr auc": 0.6534090002616602,
    "accuracy": 0.8132390319372307,
    "recall": 0.5452930728241563,
    "precision": 0.648363252375924,
    "roc auc": 0.7236543339698243,
    "f1 score": 0.592378195851423
  }
}


# GA OPTIMIZED

In [53]:
!pip install sklearn-genetic-opt


from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Continuous, Categorical, Integer
from sklearn_genetic.plots import plot_fitness_evolution, plot_search_space
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
##                     ##
## LOGISTIC REGRESSION ##
##                     ##

from sklearn.linear_model import LogisticRegression
import json

param_grid = {
    'max_iter': Integer(100, 1000),
    'penalty': Categorical(['l1', 'l2']),
    'C': Continuous(1e-6, 1e-2, distribution="log-uniform"),
    'l1_ratio': Continuous(1e-9, 1, distribution='log-uniform')
}


lr = LogisticRegression(solver="saga")
cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=lr, cv=cv,
                               scoring='accuracy', population_size=10,
                               generations=10, tournament_size=3,
                               elitism=True, crossover_probability=0.8,
                               mutation_probability=0.1, param_grid=param_grid,
                               criteria='max', algorithm='eaMuPlusLambda',
                               n_jobs=-1, verbose=True, keep_top_k=4)

In [55]:
_start =time.time()

evolved_estimator.fit(X_train, y_train)

_end = time.time()

# only in seconds

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.763781	0.00462244 	0.772983   	0.760385   
1  	18    	0.769668	0.00506463 	0.772984   	0.7619     
2  	16    	0.769559	0.00503255 	0.772984   	0.761664   
3  	19    	0.772984	1.29315e-07	0.772984   	0.772983   
4  	17    	0.772984	6.78939e-08	0.772984   	0.772984   
5  	18    	0.772984	4.9132e-08 	0.772984   	0.772984   
6  	14    	0.772984	4.29957e-08	0.772984   	0.772984   
7  	19    	0.772984	9.9501e-08 	0.772984   	0.772984   
8  	19    	0.772984	1.50743e-07	0.772984   	0.772983   
9  	17    	0.772984	7.47652e-08	0.772984   	0.772984   
10 	19    	0.772984	1.16683e-07	0.772984   	0.772983   
running for: 13m, 1s


In [56]:
print(json.dumps(analyse(evolved_estimator, X_train, y_train, X_test, y_test), indent=2))

print(evolved_estimator.best_params_)

{
  "train": {
    "pr auc": 0.4385331239116007,
    "accuracy": 0.7729834699000616,
    "recall": 0.3080289193302892,
    "precision": 0.5834234234234235,
    "roc auc": 0.6234888545350478,
    "f1 score": 0.403187647864525
  },
  "test": {
    "pr auc": 0.444185501749747,
    "accuracy": 0.7772129517073709,
    "recall": 0.316607460035524,
    "precision": 0.5991596638655462,
    "roc auc": 0.6289284388081566,
    "f1 score": 0.41429401510749564
  }
}
{'max_iter': 654, 'penalty': 'l1', 'C': 5.818752734303556e-05, 'l1_ratio': 0.0007265691017497256}


In [57]:
##                          ##
## RANDOM FOREST CLASSIFIER ##
##                          ##
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

param_grid = {
    'n_estimators': Integer(10, 100),
    'max_depth':Integer(3, 10),
    'max_leaf_nodes':Integer(1, 100),
    'max_features':Categorical(['sqrt', 'log2', None]),
    'criterion':Categorical(['gini', 'entropy']),
    'min_weight_fraction_leaf':Continuous(0.01, 0.5, distribution='log-uniform')
}

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=rf, cv=cv, scoring='accuracy',
                               population_size=10, generations=10,
                               tournament_size=3, elitism=True,
                               crossover_probability=0.8, mutation_probability=0.1,
                               param_grid=param_grid, criteria='max',
                               algorithm='eaMuPlusLambda', n_jobs=-1,
                               verbose=True, keep_top_k=4)

In [58]:
_start =time.time()

evolved_estimator.fit(X_train, y_train)

_end = time.time()

# only in seconds

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.803055	0.0282768  	0.833705   	0.751054   
1  	18    	0.825221	0.00688598 	0.833705   	0.814759   
2  	17    	0.830559	0.00250565 	0.834841   	0.827121   
3  	17    	0.833771	0.00351726 	0.837257   	0.829299   
4  	18    	0.835613	0.00174815 	0.837257   	0.83162    
5  	16    	0.838213	0.00173185 	0.840288   	0.836025   
6  	16    	0.839885	0.000943772	0.841472   	0.839009   
7  	19    	0.840435	0.000707766	0.841472   	0.83953    
8  	17    	0.840672	0.000532631	0.841519   	0.839909   
9  	20    	0.84143 	0.000813431	0.84242    	0.839199   
10 	17    	0.841998	0.000682902	0.843319   	0.841425   
running for: 7m, 11s


In [59]:
print(json.dumps(analyse(evolved_estimator, X_train, y_train, X_test, y_test), indent=2))

print(evolved_estimator.best_params_)

{
  "train": {
    "pr auc": 0.7727585974463527,
    "accuracy": 0.8440771088902572,
    "recall": 0.5795281582952816,
    "precision": 0.7378875968992248,
    "roc auc": 0.9044141926189827,
    "f1 score": 0.6491901108269394
  },
  "test": {
    "pr auc": 0.7689860182207656,
    "accuracy": 0.8480495082329539,
    "recall": 0.5932504440497336,
    "precision": 0.7442896935933148,
    "roc auc": 0.901368400958421,
    "f1 score": 0.6602421546824809
  }
}
{'n_estimators': 48, 'max_depth': 9, 'max_leaf_nodes': 49, 'max_features': None, 'criterion': 'entropy', 'min_weight_fraction_leaf': 0.01392922958037549}


In [60]:
##                     ##
## XG BOOST CLASSIFIER ##
##                     ##

from xgboost.sklearn import XGBClassifier

param_grid = {
    'n_estimators': Integer(10, 20),
    'max_depth': Integer(3, 9),
    'gamma': Continuous(1e-6, 1, distribution='log-uniform'),
    'learning_rate': Continuous(1e-6, 1e-2, distribution='log-uniform')
}


xgb = XGBClassifier();

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=xgb, cv=cv, scoring='accuracy',
                               population_size=10, generations=10,
                               tournament_size=3, elitism=True,
                               crossover_probability=0.8, mutation_probability=0.1,
                               param_grid=param_grid, criteria='max',
                               algorithm='eaMuPlusLambda', n_jobs=-1,
                               verbose=True, keep_top_k=4)

In [61]:
_start =time.time()

evolved_estimator.fit(X_train, y_train)

_end = time.time()

# only in seconds

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.844347	0.00476189 	0.849098   	0.834889   
1  	20    	0.847549	0.00154178 	0.849618   	0.845735   
2  	19    	0.848899	0.000895324	0.850376   	0.847108   
3  	17    	0.849562	0.000326604	0.850376   	0.849145   
4  	18    	0.849282	0.000659059	0.850376   	0.847724   
5  	19    	0.849514	0.000180751	0.849666   	0.84905    
6  	20    	0.849964	0.000434713	0.850708   	0.849429   
7  	18    	0.849486	0.000414982	0.850092   	0.848766   
8  	17    	0.84942 	0.000623607	0.85014    	0.84815    
9  	18    	0.849945	0.00053683 	0.850803   	0.849098   
10 	17    	0.849704	0.000676343	0.850803   	0.848766   
running for: 2m, 41s


In [62]:
print(json.dumps(analyse(evolved_estimator, X_train, y_train, X_test, y_test), indent=2))

print(evolved_estimator.best_params_)

{
  "train": {
    "pr auc": 0.76919615287568,
    "accuracy": 0.8554918770425805,
    "recall": 0.5578386605783866,
    "precision": 0.801311833834381,
    "roc auc": 0.9080040202344988,
    "f1 score": 0.6577678070667414
  },
  "test": {
    "pr auc": 0.755624521770874,
    "accuracy": 0.851696320035363,
    "recall": 0.5523978685612788,
    "precision": 0.788339670468948,
    "roc auc": 0.8993297703955172,
    "f1 score": 0.6496083550913837
  }
}
{'n_estimators': 11, 'max_depth': 7, 'gamma': 0.06904910302639553, 'learning_rate': 0.0039137188423680365}


In [63]:
##                        ##
## NAIVE BAYES CLASSIFIER ##
##                        ##


from sklearn.naive_bayes import GaussianNB
param_grid = {
    'var_smoothing': Continuous(1e-9, 1, distribution='log-uniform')
}

gnb = GaussianNB();

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=gnb, cv=cv, scoring='accuracy',
                               population_size=10, generations=10,
                               tournament_size=3, elitism=True,
                               crossover_probability=0.8, mutation_probability=0.1,
                               param_grid=param_grid, criteria='max',
                               algorithm='eaMuPlusLambda', n_jobs=-1,
                               verbose=True, keep_top_k=4)

In [64]:
_start =time.time()

evolved_estimator.fit(X_train, y_train)

_end = time.time()

# only in seconds

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')

gen	nevals	fitness	fitness_std	fitness_max	fitness_min
0  	10    	0.78607	0.00458463 	0.789703   	0.772652   
1  	17    	0.788841	0.0015545  	0.793113   	0.787477   
2  	19    	0.790186	0.00239169 	0.793113   	0.788092   
3  	19    	0.792426	0.00277125 	0.797802   	0.787193   
4  	20    	0.793743	0.00136354 	0.797802   	0.792971   
5  	20    	0.793738	0.00119501 	0.797281   	0.792924   
6  	17    	0.794368	0.00184177 	0.798702   	0.793255   
7  	18    	0.796301	0.00218925 	0.798892   	0.79354    
8  	19    	0.795661	0.00209187 	0.798702   	0.79335    
9  	19    	0.797949	0.000706223	0.798702   	0.796713   
10 	20    	0.7982  	0.000574399	0.798702   	0.796855   
running for: 0m, 34s


In [65]:
print(json.dumps(analyse(evolved_estimator, X_train, y_train, X_test, y_test), indent=2))

print(evolved_estimator.best_params_)

{
  "train": {
    "pr auc": 0.5822187924503197,
    "accuracy": 0.7992705915786482,
    "recall": 0.2996575342465753,
    "precision": 0.7387429643527205,
    "roc auc": 0.7719483093715531,
    "f1 score": 0.42636708175419596
  },
  "test": {
    "pr auc": 0.5909236040507821,
    "accuracy": 0.8027406343242347,
    "recall": 0.31127886323268206,
    "precision": 0.7497326203208556,
    "roc auc": 0.776395970325431,
    "f1 score": 0.43991214308126764
  }
}
{'var_smoothing': 0.0022022950009742415}


In [66]:
##                ##
## SVC CLASSIFIER ##
##                ##


from sklearn.svm import SVC

param_grid = {
    'C': Continuous(1e-4, 10, distribution='log-uniform'),
    'gamma': Continuous(1e-4, 10, distribution='log-uniform')
}

svc = SVC()

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=svc, cv=cv, scoring='accuracy',
                               population_size=10, generations=10,
                               tournament_size=3, elitism=True,
                               crossover_probability=0.8, mutation_probability=0.1,
                               param_grid=param_grid, criteria='max',
                               algorithm='eaMuPlusLambda', n_jobs=-1,
                               verbose=True, keep_top_k=4)

In [67]:
_start =time.time()

evolved_estimator.fit(X_train, y_train)

_end = time.time()

# only in seconds

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.776683	0.0316605  	0.820679   	0.751054   
1  	18    	0.796779	0.0302527  	0.820679   	0.751054   
2  	17    	0.822413	0.0131038  	0.847156   	0.804623   
3  	19    	0.831185	0.0132387  	0.847914   	0.820016   
4  	19    	0.841827	0.0108407  	0.847772   	0.820063   
5  	17    	0.847535	0.000267077	0.847772   	0.846966   
6  	17    	0.847776	0.000275724	0.848482   	0.84744    
7  	18    	0.848463	0.00147384 	0.852745   	0.84744    
8  	19    	0.848937	0.00200462 	0.852745   	0.84673    
9  	19    	0.851892	0.00279891 	0.854923   	0.847345   
10 	18    	0.853621	0.00122674 	0.854923   	0.851798   
running for: 27m, 10s


In [68]:
print(json.dumps(analyse(evolved_estimator, X_train, y_train, X_test, y_test), indent=2))

print(evolved_estimator.best_params_)

{
  "train": {
    "pr auc": 0.821174815849305,
    "accuracy": 0.8699379529200019,
    "recall": 0.5974124809741248,
    "precision": 0.8328912466843501,
    "roc auc": 0.9214961397762671,
    "f1 score": 0.6957677819632173
  },
  "test": {
    "pr auc": 0.7708154907790122,
    "accuracy": 0.8594319814344127,
    "recall": 0.5799289520426287,
    "precision": 0.8002450980392157,
    "roc auc": 0.8970233837883237,
    "f1 score": 0.6725025746652935
  }
}
{'C': 7.302009830075401, 'gamma': 0.04130642880082618}


In [70]:
##                ##
## ANN CLASSIFIER ##
##                ##

import random

class foo(Categorical):
    def __init__(self):
        pass
    def sample(self):
        hidden_layer_count = random.randint(1, 4)
        l = random.sample(range(8, 32), hidden_layer_count)
        return tuple(l)

param_grid = {
    'alpha': Continuous(1e-4, 0.05, distribution='log-uniform'),
    'activation': Categorical(['tanh', 'relu', 'identity', 'logistic']),
    'solver': Categorical(['lbfgs', 'sgd', 'adam']),
    'batch_size': Integer(100, 1000),
    'learning_rate': Categorical(['constant', 'invscaling', 'adaptive']),
    'hidden_layer_sizes': foo(),
    'max_iter': Integer(50, 150),
}

ann = Model()

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=ann, cv=cv, scoring='accuracy',
                               population_size=10, generations=10,
                               tournament_size=3, elitism=True,
                               crossover_probability=0.8, mutation_probability=0.1,
                               param_grid=param_grid, criteria='max',
                               algorithm='eaMuPlusLambda', n_jobs=-1,
                               verbose=True, keep_top_k=4)

In [71]:
_start =time.time()

evolved_estimator.fit(X_train, y_train)

_end = time.time()

# only in seconds

print(f'running for: {int((_end-_start)/60)}m, {int((_end-_start)%60)}s')

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.725412	0.122584   	0.819354   	0.411407   
1  	19    	0.799105	0.0194811  	0.819354   	0.751054   
2  	19    	0.808601	0.0087984  	0.819354   	0.78634    
3  	19    	0.810562	0.00485255 	0.819354   	0.802871   
4  	17    	0.8116  	0.00301351 	0.819354   	0.807891   
5  	19    	0.812372	0.00386633 	0.819354   	0.807891   
6  	19    	0.812476	0.00352791 	0.819354   	0.807891   
7  	19    	0.813508	0.00327295 	0.819354   	0.80917    
8  	16    	0.815076	0.00319923 	0.819354   	0.81097    
9  	15    	0.814261	0.00294974 	0.819354   	0.81097    
10 	19    	0.81411 	0.00231827 	0.819354   	0.811585   
running for: 35m, 39s


In [72]:
import json
print(json.dumps(analyseANN(evolved_estimator, X_train, y_train, X_test, y_test), indent=2))

print(evolved_estimator.best_params_)

{
  "train": {
    "pr auc": 0.666485621454837,
    "accuracy": 0.8209160233031781,
    "recall": 0.5490867579908676,
    "precision": 0.6716313707237608,
    "roc auc": 0.7300519871810931,
    "f1 score": 0.6042081021668585
  },
  "test": {
    "pr auc": 0.6730914091221867,
    "accuracy": 0.8237374295502266,
    "recall": 0.5617229129662522,
    "precision": 0.6753870795515217,
    "roc auc": 0.7361358422415489,
    "f1 score": 0.6133333333333333
  }
}
{'alpha': 0.004922607390769823, 'activation': 'logistic', 'solver': 'adam', 'batch_size': 460, 'learning_rate': 'constant', 'hidden_layer_sizes': (27, 24, 18), 'max_iter': 125}
